Credit: https://towardsdatascience.com/using-python-to-create-a-world-map-from-a-list-of-country-names-cd7480d03b10

In [40]:
import pandas as pd
import numpy as np
import igraph as ig
import collections
import matplotlib
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
import json
%matplotlib inline

In [41]:
df_spinglass = pd.read_csv('df_spinglass.csv')
df_spinglass

,Unnamed: 0,name,community
0,0,Albania,0
1,1,Bahamas,0
2,2,Algeria,5
3,3,British Virgin Islands,4
4,4,United Kingdom,5
...,...,...,...
191,191,Venezuela,3
192,192,Vietnam,4
193,193,Yemen,3
194,194,Zambia,4


In [42]:
df_walktrap = pd.read_csv('df_walktrap.csv')
df_walktrap

,Unnamed: 0,name,community
0,0,Albania,0
1,1,Bahamas,1
2,2,Algeria,2
3,3,British Virgin Islands,1
4,4,United Kingdom,1
...,...,...,...
191,191,Venezuela,1
192,192,Vietnam,1
193,193,Yemen,51
194,194,Zambia,52


In [43]:
df_infomap = pd.read_csv('df_infomap.csv')
df_infomap

,Unnamed: 0,name,community
0,0,Albania,0
1,1,Bahamas,0
2,2,Algeria,0
3,3,British Virgin Islands,0
4,4,United Kingdom,0
...,...,...,...
191,191,Venezuela,0
192,192,Vietnam,0
193,193,Yemen,0
194,194,Zambia,0


In [44]:
df_leading_eigen = pd.read_csv("df_leading_eigenvector.csv")
df_leading_eigen

,Unnamed: 0,name,community
0,0,Albania,0
1,1,Bahamas,0
2,2,Algeria,1
3,3,British Virgin Islands,1
4,4,United Kingdom,1
...,...,...,...
191,191,Venezuela,0
192,192,Vietnam,1
193,193,Yemen,0
194,194,Zambia,1


In [45]:
def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)

In [46]:
df_spinglass['codes'] = df_spinglass['name'].apply(get_continent)
df_spinglass['country_code'] = df_spinglass['codes'].apply(lambda x: x[0])
df_spinglass['continent_code'] = df_spinglass['codes'].apply(lambda x: x[1])

df_walktrap['codes'] = df_walktrap['name'].apply(get_continent)
df_walktrap['country_code'] = df_walktrap['codes'].apply(lambda x: x[0])
df_walktrap['continent_code'] = df_walktrap['codes'].apply(lambda x: x[1])

df_infomap['codes'] = df_infomap['name'].apply(get_continent)
df_infomap['country_code'] = df_infomap['codes'].apply(lambda x: x[0])
df_infomap['continent_code'] = df_infomap['codes'].apply(lambda x: x[1])

df_leading_eigen['codes'] = df_leading_eigen['name'].apply(get_continent)
df_leading_eigen['country_code'] = df_leading_eigen['codes'].apply(lambda x: x[0])
df_leading_eigen['continent_code'] = df_leading_eigen['codes'].apply(lambda x: x[1])

df_spinglass
#df_walktrap
#df_infomap

,Unnamed: 0,name,community,codes,country_code,continent_code
0,0,Albania,0,"(AL, EU)",AL,EU
1,1,Bahamas,0,"(BS, NA)",BS,NA
2,2,Algeria,5,"(DZ, AF)",DZ,AF
3,3,British Virgin Islands,4,"(VG, NA)",VG,NA
4,4,United Kingdom,5,"(GB, EU)",GB,EU
...,...,...,...,...,...,...
191,191,Venezuela,3,"(VE, SA)",VE,SA
192,192,Vietnam,4,"(VN, AS)",VN,AS
193,193,Yemen,3,"(YE, AS)",YE,AS
194,194,Zambia,4,"(ZM, AF)",ZM,AF


In [47]:
df_countries = df_spinglass[['name','country_code']]
df_countries

,name,country_code
0,Albania,AL
1,Bahamas,BS
2,Algeria,DZ
3,British Virgin Islands,VG
4,United Kingdom,GB
...,...,...
191,Venezuela,VE
192,Vietnam,VN
193,Yemen,YE
194,Zambia,ZM


In [48]:
def geolocate(country,country_code):
    geolocator = Nominatim(user_agent = country)
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country, country_codes = country_code)
        # And return latitude and longitude
        return (round(loc.latitude,2), round(loc.longitude,2))
    except:
        # Return missing value
        loc = geolocator.geocode(country, country_codes = None)
        return (round(loc.latitude,2), round(loc.longitude,2))

In [49]:
df_countries['Lat_Long'] = df_countries.apply(lambda x: geolocate(x['name'], x['country_code']),axis = 1)

<ipython-input-49-a7c0add9ac3a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_countries['Lat_Long'] = df_countries.apply(lambda x: geolocate(x['name'], x['country_code']),axis = 1)


In [50]:
df_countries['Latitude'] = df_countries['Lat_Long'].apply(lambda x: x[0])
df_countries['Longitude'] = df_countries['Lat_Long'].apply(lambda x: x[1])

df_spinglass = df_spinglass.merge(df_countries, how = 'left')
df_walktrap = df_walktrap.merge(df_countries, how = 'left')
df_infomap = df_infomap.merge(df_countries, how = 'left')
df_leading_eigen = df_leading_eigen.merge(df_countries,how = 'left')
df_walktrap

<ipython-input-50-8e766aabb196>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_countries['Latitude'] = df_countries['Lat_Long'].apply(lambda x: x[0])
<ipython-input-50-8e766aabb196>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_countries['Longitude'] = df_countries['Lat_Long'].apply(lambda x: x[1])


,Unnamed: 0,name,community,codes,country_code,continent_code,Lat_Long,Latitude,Longitude
0,0,Albania,0,"(AL, EU)",AL,EU,"(41.0, 20.0)",41.00,20.00
1,1,Bahamas,1,"(BS, NA)",BS,NA,"(24.77, -78.0)",24.77,-78.00
2,2,Algeria,2,"(DZ, AF)",DZ,AF,"(28.0, 3.0)",28.00,3.00
3,3,British Virgin Islands,1,"(VG, NA)",VG,NA,"(18.4, -64.57)",18.40,-64.57
4,4,United Kingdom,1,"(GB, EU)",GB,EU,"(54.7, -3.28)",54.70,-3.28
...,...,...,...,...,...,...,...,...,...
191,191,Venezuela,1,"(VE, SA)",VE,SA,"(8.0, -66.11)",8.00,-66.11
192,192,Vietnam,1,"(VN, AS)",VN,AS,"(13.29, 108.43)",13.29,108.43
193,193,Yemen,51,"(YE, AS)",YE,AS,"(16.35, 47.89)",16.35,47.89
194,194,Zambia,52,"(ZM, AF)",ZM,AF,"(-14.52, 27.56)",-14.52,27.56


In [51]:
df_spinglass.dropna(inplace = True)
df_spinglass.reset_index(drop = True, inplace = True)

df_walktrap.dropna(inplace = True)
df_walktrap.reset_index(drop = True, inplace = True)

df_infomap.dropna(inplace = True)
df_infomap.reset_index(drop = True, inplace = True)

df_leading_eigen.dropna(inplace = True)
df_leading_eigen.reset_index(drop = True, inplace = True)

df_spinglass

,Unnamed: 0,name,community,codes,country_code,continent_code,Lat_Long,Latitude,Longitude
0,0,Albania,0,"(AL, EU)",AL,EU,"(41.0, 20.0)",41.00,20.00
1,1,Bahamas,0,"(BS, NA)",BS,NA,"(24.77, -78.0)",24.77,-78.00
2,2,Algeria,5,"(DZ, AF)",DZ,AF,"(28.0, 3.0)",28.00,3.00
3,3,British Virgin Islands,4,"(VG, NA)",VG,NA,"(18.4, -64.57)",18.40,-64.57
4,4,United Kingdom,5,"(GB, EU)",GB,EU,"(54.7, -3.28)",54.70,-3.28
...,...,...,...,...,...,...,...,...,...
191,191,Venezuela,3,"(VE, SA)",VE,SA,"(8.0, -66.11)",8.00,-66.11
192,192,Vietnam,4,"(VN, AS)",VN,AS,"(13.29, 108.43)",13.29,108.43
193,193,Yemen,3,"(YE, AS)",YE,AS,"(16.35, 47.89)",16.35,47.89
194,194,Zambia,4,"(ZM, AF)",ZM,AF,"(-14.52, 27.56)",-14.52,27.56


In [52]:
df_spinglass['Latitude'] = df_spinglass['Lat_Long'].apply(lambda x: x[0])
df_spinglass['Longitude'] = df_spinglass['Lat_Long'].apply(lambda x: x[1])

df_walktrap['Latitude'] = df_walktrap['Lat_Long'].apply(lambda x: x[0])
df_walktrap['Longitude'] = df_walktrap['Lat_Long'].apply(lambda x: x[1])

df_infomap['Latitude'] = df_infomap['Lat_Long'].apply(lambda x: x[0])
df_infomap['Longitude'] = df_infomap['Lat_Long'].apply(lambda x: x[1])

df_leading_eigen['Latitude'] = df_leading_eigen['Lat_Long'].apply(lambda x: x[0])
df_leading_eigen['Longitude'] = df_leading_eigen['Lat_Long'].apply(lambda x: x[1])

df_spinglass

,Unnamed: 0,name,community,codes,country_code,continent_code,Lat_Long,Latitude,Longitude
0,0,Albania,0,"(AL, EU)",AL,EU,"(41.0, 20.0)",41.00,20.00
1,1,Bahamas,0,"(BS, NA)",BS,NA,"(24.77, -78.0)",24.77,-78.00
2,2,Algeria,5,"(DZ, AF)",DZ,AF,"(28.0, 3.0)",28.00,3.00
3,3,British Virgin Islands,4,"(VG, NA)",VG,NA,"(18.4, -64.57)",18.40,-64.57
4,4,United Kingdom,5,"(GB, EU)",GB,EU,"(54.7, -3.28)",54.70,-3.28
...,...,...,...,...,...,...,...,...,...
191,191,Venezuela,3,"(VE, SA)",VE,SA,"(8.0, -66.11)",8.00,-66.11
192,192,Vietnam,4,"(VN, AS)",VN,AS,"(13.29, 108.43)",13.29,108.43
193,193,Yemen,3,"(YE, AS)",YE,AS,"(16.35, 47.89)",16.35,47.89
194,194,Zambia,4,"(ZM, AF)",ZM,AF,"(-14.52, 27.56)",-14.52,27.56


In [53]:
df_spinglass['community'].value_counts()

4    77
5    45
3    38
0    19
1    15
2     2
Name: community, dtype: int64

In [54]:
df_walktrap['community'].value_counts()

1     137
4       8
52      1
13      1
23      1
22      1
21      1
20      1
19      1
18      1
17      1
16      1
15      1
14      1
12      1
25      1
11      1
10      1
9       1
8       1
7       1
6       1
5       1
3       1
2       1
24      1
26      1
51      1
27      1
50      1
49      1
48      1
47      1
46      1
45      1
44      1
43      1
42      1
41      1
40      1
39      1
38      1
37      1
36      1
35      1
34      1
33      1
32      1
31      1
30      1
29      1
28      1
0       1
Name: community, dtype: int64

In [55]:
df_spinglass['community'].value_counts()

4    77
5    45
3    38
0    19
1    15
2     2
Name: community, dtype: int64

## Let's map the community for Spinglass

In [56]:
#Setting up the world countries data URL
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [57]:
#empty map
world_map_spinglass = folium.Map(tiles="cartodbpositron")
marker_cluster_spinglass = MarkerCluster().add_to(world_map_spinglass)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_spinglass)):
        lat = df_spinglass.iloc[i]['Latitude']
        long = df_spinglass.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_spinglass.iloc[i]['name'],
                                   df_spinglass.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_spinglass)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Spinglass Community Choropleth',
    data=df_spinglass,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set1',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_spinglass)
world_map_spinglass

In [58]:
world_map_spinglass.save('world_map_spinglass.html')

## Walktrap

In [59]:
#empty map
world_map_walktrap = folium.Map(tiles="cartodbpositron")
marker_cluster_walktrap = MarkerCluster().add_to(world_map_walktrap)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_walktrap)):
        lat = df_walktrap.iloc[i]['Latitude']
        long = df_walktrap.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_walktrap.iloc[i]['name'],
                                   df_walktrap.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_walktrap)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Walktrap Community Choropleth',
    data=df_walktrap,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set1',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_walktrap)
world_map_walktrap

In [60]:
world_map_walktrap.save('world_map_walktrap.html')

## Infomap

In [61]:
#empty map
world_map_infomap = folium.Map(tiles="cartodbpositron")
marker_cluster_infomap = MarkerCluster().add_to(world_map_infomap)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_infomap)):
        lat = df_infomap.iloc[i]['Latitude']
        long = df_infomap.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_infomap.iloc[i]['name'],
                                   df_infomap.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_infomap)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Infomap Community Choropleth',
    data=df_infomap,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set1',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_infomap)
world_map_infomap

In [62]:
world_map_infomap.save('world_map_infomap.html')

## Leading Eigen

In [63]:
#empty map
world_map_eigen = folium.Map(tiles="cartodbpositron")
marker_cluster_eigen = MarkerCluster().add_to(world_map_eigen)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_leading_eigen)):
        lat = df_leading_eigen.iloc[i]['Latitude']
        long = df_leading_eigen.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_leading_eigen.iloc[i]['name'],
                                   df_leading_eigen.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_eigen)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Eigen Community Choropleth',
    data=df_leading_eigen,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set1',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_eigen)
world_map_eigen